In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo


In [28]:
from Bio import SeqIO
from tqdm import tqdm

In [90]:

# File path to your FASTA file
fastaPath = '/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta'   # <--- substitute by your local path

# Open file with "with" statement to avoid problems with access 
# to original file (in case computer hangs
# or there will be any other problem)
def read_fasta(fastaPath):
    with open(path_to_file, mode='r') as handle:
        for record in SeqIO.parse(handle, 'fasta'):
            identifier = record.id
            description = record.description
            sequence = record.seq
    data = {'ID':record.id, 'Sequences': record.seq, 'description': record.description}        
    return pd.DataFrame(data)    
    

In [91]:
def get_top_go_terms(data, num_terms):
    # Calculate term frequencies
    term_counts = data['term'].value_counts()
    freq_counts = term_counts / len(data)
    
    # Select top terms based on frequency
    freq_top = freq_counts.nlargest(num_terms)
    
    # Return the top terms
    return freq_top


In [92]:
train_terms=pd.read_csv('/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv',sep='\t')
top_terms = get_top_go_terms(train_terms, 10)
top_terms

GO:0005575    0.017322
GO:0008150    0.017191
GO:0110165    0.017019
GO:0003674    0.014661
GO:0005622    0.013197
GO:0009987    0.011427
GO:0043226    0.011351
GO:0043229    0.010872
GO:0005488    0.010698
GO:0043227    0.010338
Name: term, dtype: float64

In [93]:
test_data = read_fasta('/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta')

# Perform predictions for each test sequence using the top terms
results =[]
for index, row in tqdm(test_data.iterrows(), total=test_data.shape[0], position=0):
    for term, freq in top_terms.items():
        results.append((row['ID'], term, freq))

# Create a DataFrame from the results and save it to a TSV file
final_results = pd.DataFrame(results, columns=['Id', 'GO term', 'Confidence'])
final_results.to_csv('submission.tsv', sep='\t', index=False)

100%|██████████| 138/138 [00:00<00:00, 7420.31it/s]


In [95]:
submission = pd.read_csv('/kaggle/input/cafa-5-submission/submission.tsv',
    sep='\t', header=None)

submission.to_csv('submission.tsv',
    sep='\t', header=False, index=False)